# Procesamiento y limpieza de los datos

In [ ]:
import datetime as dt
import pandas as pd
import pathlib
import os
import time
# os.chdir(pathlib.Path.cwd())

# Conexión de google drive con el repo
from google.colab import drive
drive.mount('/content/drive')

###==========================
ruta = os.path.join("/content/drive/MyDrive","MonografiaUdeA")
os.chdir(ruta)
abspath = pathlib.Path.cwd()
print(f"Directorio de trabajo: {abspath}")
###==========================

if not os.path.exists(os.path.join(abspath, "datasets", "procesado")):
    os.makedirs(os.path.join(abspath, "datasets", "procesado"))

Mounted at /content/drive
Directorio de trabajo: /content/drive/MyDrive/MonografiaUdeA


## Aportes hídricos

In [ ]:
aportes = pd.read_csv(os.path.join(abspath,"datasets","AporEner.csv"))
aportes = aportes[['Date','Value']]
aportes = aportes.rename(columns = {'Date': 'fecha',
                                    'Value': 'aportes_energia'}).sort_values(by = 'fecha', ascending = True)

aportes['fecha'] = pd.to_datetime(aportes['fecha'])
aportes = aportes.reset_index(drop=True)
aportes.to_csv(os.path.join(abspath,"datasets", "procesado","aportes.csv"), index=False)
aportes.head(3)

,fecha,aportes_energia
0,2010-01-01,45616100.0
1,2010-01-02,43097100.0
2,2010-01-03,43400400.0


## Compras en bolsa

In [ ]:
hours = [f"Values_Hour{str(i).zfill(2)}" for i in range(1,25)]
compras_bolsa = pd.read_csv(os.path.join(abspath,"datasets","CompBolsNaciEner.csv"))

compras_bolsa['compras_bolsa'] = compras_bolsa[hours].sum(axis=1)
compras_bolsa = compras_bolsa[ ['Date'] + ['compras_bolsa']]

compras_bolsa = compras_bolsa.rename(columns = {'Date': 'fecha'}).sort_values(by = 'fecha', ascending = True)
compras_bolsa['fecha'] = pd.to_datetime(compras_bolsa['fecha'])
compras_bolsa = compras_bolsa.reset_index(drop=True)
compras_bolsa.to_csv(os.path.join(abspath,"datasets", "procesado","compras_bolsa.csv"), index=False)
compras_bolsa.head()

,fecha,compras_bolsa
0,2010-01-01,44877553.69
1,2010-01-02,43958164.96
2,2010-01-03,53146284.88
3,2010-01-04,46780069.04
4,2010-01-05,47311962.44


## Compras en contratos

In [ ]:
#%%
compras_contrato = pd.read_csv(os.path.join(abspath,"datasets","CompContEner.csv"))
compras_contrato['compras_contrato'] = compras_contrato[hours].sum(axis=1)
compras_contrato = compras_contrato[ ['Date'] + ['compras_contrato']]

compras_contrato = compras_contrato.rename(columns = {'Date': 'fecha'}).sort_values(by = 'fecha', ascending = True)
compras_contrato['fecha'] = pd.to_datetime(compras_contrato['fecha'])
compras_contrato.to_csv(os.path.join(abspath,"datasets", "procesado","compras_contrato.csv"), index=False)

## Demanda

In [ ]:
demanda = pd.read_csv(os.path.join(abspath,"datasets","DemaReal.csv"))
demanda['demanda'] = demanda[hours].sum(axis=1)
# demanda = demanda[ ['Date'] + ['demanda']]
demanda = demanda[['Date','demanda']]


demanda = demanda.rename(columns = {'Date': 'fecha',
                                    'demanda': 'demanda'}).sort_values(by = 'fecha', ascending = True)
demanda['fecha'] = pd.to_datetime(demanda['fecha'])
demanda = demanda.reset_index(drop=True)
demanda.to_csv(os.path.join(abspath,"datasets", "procesado","demanda.csv"), index=False)
demanda
# demanda.to_csv('demanda_final.csv', index=False)

,fecha,demanda
0,2010-01-01,1.133616e+08
1,2010-01-02,1.254718e+08
2,2010-01-03,1.211452e+08
3,2010-01-04,1.439317e+08
4,2010-01-05,1.498691e+08
...,...,...
3647,2019-12-27,1.982598e+08
3648,2019-12-28,1.921742e+08
3649,2019-12-29,1.796031e+08
3650,2019-12-30,1.902380e+08


## Listado de Recursos de Generación

In [ ]:
recursos = pd.read_csv(os.path.join(abspath,"datasets","listado_recursos.csv"))

recursos = recursos[['Values_Code', 'Values_Type', 'Values_EnerSource', 'Values_State']]
recursos = recursos.rename(columns = {'Values_Code': 'codigo_recurso',
                                      'Values_Type': 'tipo_generacion',
                                      'Values_EnerSource':'combustible',
                                      'Values_State':'estado'})
# recursos['fecha'] = pd.to_datetime(recursos['fecha'])
# recursos = recursos.reset_index(drop=True)
recursos.to_csv(os.path.join(abspath,"datasets", "procesado","listado_recursos.csv"), index=False)
print(recursos.columns)
recursos.head(3)

Index(['codigo_recurso', 'tipo_generacion', 'combustible', 'estado'], dtype='object')


,codigo_recurso,tipo_generacion,combustible,estado
0,2QBW,HIDRAULICA,AGUA,OPERACION
1,2QEK,HIDRAULICA,AGUA,OPERACION
2,2QRL,HIDRAULICA,AGUA,OPERACION


## Disponibilidad declarada

In [ ]:
#%%
def tipo_gen(row):
    if row['tipo_generacion'] == 'TERMICA':
        return 'TERMICA'

    elif row['tipo_generacion'] == 'HIDRAULICA':
        return 'HIDRAULICA'
    elif row['tipo_generacion'] != 'TERMICA':
        return 'NO TERMICA'

# Crear la nueva columna basada en la función condicion
# disponibilidad_declarada = pd.read_csv('disponibilidad_declarada.csv')
disponibilidad_declarada = pd.read_csv(os.path.join(abspath,"datasets","DispoDeclarada.csv"))
disponibilidad_declarada['disponibilidad_declarada'] = disponibilidad_declarada[hours].mean(axis=1)
disponibilidad_declarada = pd.merge(disponibilidad_declarada, recursos, how = 'left', left_on = 'Values_code', right_on = 'codigo_recurso' )
disponibilidad_declarada_total = disponibilidad_declarada.groupby(['Date'])['disponibilidad_declarada'].mean().reset_index()


disponibilidad_declarada['tipo_generacion_aux'] = disponibilidad_declarada.apply(lambda row: tipo_gen(row), axis=1)


disponibilidad_declarada = disponibilidad_declarada.groupby(['Date','tipo_generacion_aux'])['disponibilidad_declarada'].mean().reset_index()
# disponibilidad_declarada = disponibilidad_declarada[disponibilidad_declarada['tipo_generacion_aux'].isin(['HIDRAULICA', 'TERMICA'])]
disponibilidad_declarada = pd.pivot_table(disponibilidad_declarada, index = 'Date', columns = 'tipo_generacion_aux', values = 'disponibilidad_declarada').reset_index()
disponibilidad_declarada = disponibilidad_declarada.rename(columns = {
                                                                      'Date': 'fecha',
                                                                      'HIDRAULICA': 'disp_dec_hidraulica',
                                                                      'TERMICA': 'disp_dec_termica',
                                                                      'NO TERMICA': 'disp_dec_no_termica'})

disponibilidad_declarada_total = disponibilidad_declarada_total.rename(columns = {
                                                                      'Date': 'fecha'})
disponibilidad_declarada['fecha'] = pd.to_datetime(disponibilidad_declarada['fecha'])
disponibilidad_declarada_total['fecha'] = pd.to_datetime(disponibilidad_declarada_total['fecha'])
disponibilidad_declarada = pd.merge(disponibilidad_declarada, disponibilidad_declarada_total, how = 'left', on = 'fecha')
disponibilidad_declarada.to_csv(os.path.join(abspath,"datasets", "procesado","disponibilidad_declarada.csv"), index=False)
disponibilidad_declarada.head(3)

,fecha,disp_dec_hidraulica,disp_dec_no_termica,disp_dec_termica,disponibilidad_declarada
0,2010-01-01,356125.000000,102616.666667,121611.607143,213690.151515
1,2010-01-02,353776.515152,102616.666667,121633.928571,212762.121212
2,2010-01-03,356579.545455,102616.666667,119455.357143,212774.242424


## Generación

In [ ]:
#%%
# Crear la nueva columna basada en la función condicion
# generacion = pd.read_csv('generacion.csv')
generacion = pd.read_csv(os.path.join(abspath,"datasets","Gene.csv"))
generacion['generacion'] = generacion[hours].sum(axis=1)
generacion = pd.merge(generacion, recursos, how = 'left', left_on = 'Values_code', right_on = 'codigo_recurso' )
generacion_total = generacion.groupby(['Date'])['generacion'].sum().reset_index()


generacion['tipo_generacion_aux'] = generacion.apply(lambda row: tipo_gen(row), axis=1)


generacion = generacion.groupby(['Date','tipo_generacion_aux'])['generacion'].sum().reset_index()
# generacion = generacion[generacion['tipo_generacion_aux'].isin(['HIDRAULICA', 'TERMICA'])]
generacion = pd.pivot_table(generacion, index = 'Date', columns = 'tipo_generacion_aux', values = 'generacion').reset_index()
generacion = generacion.rename(columns = {
                                                                      'Date': 'fecha',
                                                                      'HIDRAULICA': 'gen_hidraulica',
                                                                      'TERMICA': 'gen_termica',
                                                                      'NO TERMICA': 'gen_no_termica'})

generacion_total = generacion_total.rename(columns = {
                                                                      'Date': 'fecha'})
generacion['fecha'] = pd.to_datetime(generacion['fecha'])
generacion_total['fecha'] = pd.to_datetime(generacion_total['fecha'])
generacion = pd.merge(generacion, generacion_total, how = 'left', on = 'fecha')
generacion.to_csv(os.path.join(abspath,"datasets", "procesado","generacion.csv"), index=False)
generacion.head(3)

,fecha,gen_hidraulica,gen_no_termica,gen_termica,generacion
0,2010-01-01,35796804.67,13362226.83,66260052.96,1.154191e+08
1,2010-01-02,46595646.99,13393741.63,67691806.16,1.276812e+08
2,2010-01-03,42578875.20,13046074.83,68012828.41,1.236378e+08


## Precio de bolsa

In [ ]:
# disponibilidad_real = pd.read_csv('disponibilidad_real.csv')
precio_bolsa = pd.read_csv(os.path.join(abspath,"datasets","PrecBolsNaci.csv"))
precio_bolsa['precio_bolsa'] = precio_bolsa[hours].mean(axis=1)
precio_bolsa = precio_bolsa.groupby('Date')['precio_bolsa'].mean().reset_index()
precio_bolsa = precio_bolsa[ ['Date'] + ['precio_bolsa']]
precio_bolsa = precio_bolsa.rename(columns = {'Date': 'fecha'}).sort_values(by = 'fecha', ascending = True).reset_index(drop = True)
precio_bolsa['fecha'] = pd.to_datetime(precio_bolsa['fecha'])
precio_bolsa.to_csv(os.path.join(abspath,"datasets", "procesado","precio_bolsa.csv"), index=False)
precio_bolsa.head()

,fecha,precio_bolsa
0,2010-01-01,120.501228
1,2010-01-02,118.495340
2,2010-01-03,121.160145
3,2010-01-04,123.962937
4,2010-01-05,120.666437


## Precio de oferta

In [ ]:
# disponibilidad_real = pd.read_csv('disponibilidad_real.csv')
precio_oferta = pd.read_csv(os.path.join(abspath,"datasets","PrecOferDesp.csv"))
precio_oferta['precio_oferta'] = precio_oferta[hours].mean(axis=1)
precio_oferta = precio_oferta.groupby('Date')['precio_oferta'].mean().reset_index()
precio_oferta = precio_oferta[ ['Date'] + ['precio_oferta']]
precio_oferta = precio_oferta.rename(columns = {'Date': 'fecha'}).sort_values(by = 'fecha', ascending = True).reset_index(drop = True)
precio_oferta['fecha'] = pd.to_datetime(precio_oferta['fecha'])
precio_oferta.to_csv(os.path.join(abspath,"datasets", "procesado","precio_oferta.csv"), index=False)
precio_oferta.head()

,fecha,precio_oferta
0,2010-01-01,267.437633
1,2010-01-02,277.153592
2,2010-01-03,313.552755
3,2010-01-04,303.519878
4,2010-01-05,339.600449


## Ventas en bolsa nacional

In [ ]:
hours = [f"Values_Hour{str(i).zfill(2)}" for i in range(1,25)]
ventas_bolsa = pd.read_csv(os.path.join(abspath,"datasets","VentBolsNaciEner.csv"))

ventas_bolsa['ventas_bolsa'] = ventas_bolsa[hours].sum(axis=1)
ventas_bolsa = ventas_bolsa[ ['Date'] + ['ventas_bolsa']]

ventas_bolsa = ventas_bolsa.rename(columns = {'Date': 'fecha'}).sort_values(by = 'fecha', ascending = True)
ventas_bolsa['fecha'] = pd.to_datetime(ventas_bolsa['fecha'])
ventas_bolsa = ventas_bolsa.reset_index(drop=True)
ventas_bolsa.to_csv(os.path.join(abspath,"datasets", "procesado","ventas_bolsa.csv"), index=False)
ventas_bolsa.head()

,fecha,ventas_bolsa
0,2010-01-01,44877553.69
1,2010-01-02,43958164.96
2,2010-01-03,53146284.87
3,2010-01-04,46759694.44
4,2010-01-05,47302938.66


## Ventas en contratos

In [ ]:
hours = [f"Values_Hour{str(i).zfill(2)}" for i in range(1,25)]
ventas_contrato = pd.read_csv(os.path.join(abspath,"datasets","VentContEner.csv"))

ventas_contrato['ventas_contrato'] = ventas_contrato[hours].sum(axis=1)
ventas_contrato = ventas_contrato[ ['Date'] + ['ventas_contrato']]

ventas_contrato = ventas_contrato.rename(columns = {'Date': 'fecha'}).sort_values(by = 'fecha', ascending = True)
ventas_contrato['fecha'] = pd.to_datetime(ventas_contrato['fecha'])
ventas_contrato = ventas_contrato.reset_index(drop=True)
ventas_contrato.to_csv(os.path.join(abspath,"datasets", "procesado","ventas_contrato.csv"), index=False)
ventas_contrato.head()

,fecha,ventas_contrato
0,2010-01-01,1.395132e+08
1,2010-01-02,1.490199e+08
2,2010-01-03,1.427063e+08
3,2010-01-04,1.623375e+08
4,2010-01-05,1.644152e+08


## Volumen útil

In [ ]:
vol_util = pd.read_csv(os.path.join(abspath,"datasets","VoluUtilDiarEner.csv"))
# vol_util['vol_util'] = vol_util[hours].sum(axis=1)
# vol_util = vol_util[ ['Date'] + ['vol_util']]
vol_util = vol_util[['Date','Value']]


vol_util = vol_util.rename(columns = {'Date': 'fecha',
                                    'Value': 'vol_util'}).sort_values(by = 'fecha', ascending = True)
vol_util['fecha'] = pd.to_datetime(vol_util['fecha'])
vol_util = vol_util.reset_index(drop=True)
vol_util.to_csv(os.path.join(abspath,"datasets", "procesado","vol_util.csv"), index=False)
vol_util

,fecha,vol_util
0,2010-01-01,1.000661e+10
1,2010-01-02,9.995147e+09
2,2010-01-03,9.992677e+09
3,2010-01-04,9.965341e+09
4,2010-01-05,9.926760e+09
...,...,...
3647,2019-12-27,1.117787e+10
3648,2019-12-28,1.112934e+10
3649,2019-12-29,1.108964e+10
3650,2019-12-30,1.107393e+10


## Vertimientos

In [ ]:
vertimientos = pd.read_csv(os.path.join(abspath,"datasets","VertEner.csv"))
# vertimientos['vertimientos'] = vertimientos[hours].sum(axis=1)
# vertimientos = vertimientos[ ['Date'] + ['vertimientos']]
vertimientos = vertimientos[['Date','Value']]


vertimientos = vertimientos.rename(columns = {'Date': 'fecha',
                                    'Value': 'vertimientos'}).sort_values(by = 'fecha', ascending = True)
vertimientos['fecha'] = pd.to_datetime(vertimientos['fecha'])
vertimientos = vertimientos.reset_index(drop=True)
vertimientos.to_csv(os.path.join(abspath,"datasets", "procesado","vertimientos.csv"), index=False)
vertimientos

,fecha,vertimientos
0,2010-02-15,4.259369e+03
1,2010-02-22,6.366273e+05
2,2010-02-23,7.226934e+05
3,2010-02-24,8.421727e+05
4,2010-02-25,8.327154e+05
...,...,...
1977,2019-11-30,2.300867e+07
1978,2019-12-01,1.408607e+07
1979,2019-12-02,3.604687e+06
1980,2019-12-17,3.153345e+05


## Indice ONI

In [ ]:
ONI = pd.read_excel(os.path.join(abspath,"datasets","Indice_ONI.xlsx"))
indice_ONI = pd.melt(ONI, id_vars=['Year'], var_name='Month', value_name='indice_ONI')
indice_ONI['fecha'] = pd.to_datetime(indice_ONI['Year'].astype(str) + '-' + indice_ONI['Month'], format='%Y-%m')
indice_ONI.drop(['Year', 'Month'], axis=1, inplace=True)
indice_ONI.sort_values(by='fecha', inplace=True)
indice_ONI.set_index('fecha', inplace=True)
indice_ONI = indice_ONI.resample('D').asfreq().ffill()
indice_ONI = indice_ONI.reset_index()
indice_ONI.to_csv(os.path.join(abspath,"datasets", "procesado","Indice_ONI.csv"), index=False)
indice_ONI.head()
# Establecer Date como índice

,fecha,indice_ONI
0,2000-01-01,-1.7
1,2000-01-02,-1.7
2,2000-01-03,-1.7
3,2000-01-04,-1.7
4,2000-01-05,-1.7


# Unión dataset

In [ ]:
join_type = 'left'
df_final = pd.merge(precio_bolsa, demanda, how = join_type, on = 'fecha')
# df_final = pd.merge(df_final, compras_contrato, how = join_type, on = 'fecha')
# df_final = pd.merge(df_final, compras_bolsa, how = join_type, on = 'fecha')
df_final = pd.merge(df_final, disponibilidad_declarada, how = join_type, on = 'fecha')
# df_final = pd.merge(df_final, disponibilidad_real, how = join_type, on = 'fecha')
df_final = pd.merge(df_final, generacion, how = join_type, on = 'fecha')
df_final = pd.merge(df_final, aportes, how = join_type, on = 'fecha')
df_final = pd.merge(df_final, precio_oferta, how = join_type, on = 'fecha')
# df_final = pd.merge(df_final, ventas_bolsa, how = join_type, on = 'fecha')
# df_final = pd.merge(df_final, ventas_contrato, how = join_type, on = 'fecha')
df_final = pd.merge(df_final, vol_util, how = join_type, on = 'fecha')
df_final = pd.merge(df_final, indice_ONI, how = join_type, on = 'fecha')
df_final = pd.merge(df_final, vertimientos, how = 'left', on = 'fecha')
df_final.to_csv(os.path.join(abspath,"datasets", "procesado","1_dataset_unificado.csv"), index=False) # .H5
df_final.head(5)

,fecha,precio_bolsa,demanda,disp_dec_hidraulica,disp_dec_no_termica,disp_dec_termica,disponibilidad_declarada,gen_hidraulica,gen_no_termica,gen_termica,generacion,aportes_energia,precio_oferta,vol_util,indice_ONI,vertimientos
0,2010-01-01,120.501228,1.133616e+08,356125.000000,102616.666667,121611.607143,213690.151515,35796804.67,13362226.83,66260052.96,1.154191e+08,45616100.0,267.437633,1.000661e+10,1.5,NaN
1,2010-01-02,118.495340,1.254718e+08,353776.515152,102616.666667,121633.928571,212762.121212,46595646.99,13393741.63,67691806.16,1.276812e+08,43097100.0,277.153592,9.995147e+09,1.5,NaN
2,2010-01-03,121.160145,1.211452e+08,356579.545455,102616.666667,119455.357143,212774.242424,42578875.20,13046074.83,68012828.41,1.236378e+08,43400400.0,313.552755,9.992677e+09,1.5,NaN
3,2010-01-04,123.962937,1.439317e+08,341617.424242,102616.666667,116611.607143,205341.666667,62242326.79,13160643.95,70554488.25,1.459575e+08,40410400.0,303.519878,9.965341e+09,1.5,NaN
4,2010-01-05,120.666437,1.498691e+08,346125.000000,102616.666667,116613.095238,207145.454545,69173462.65,14514419.15,68460147.29,1.521480e+08,41190600.0,339.600449,9.926760e+09,1.5,NaN


In [ ]:
print("Dimensiones", df_final.shape)

Dimensiones (3652, 16)
